In [68]:
from speechbrain.dataio.dataio import read_audio, write_audio
import torch
from glob import glob
from speechbrain.augment.time_domain import AddNoise
from tqdm import tqdm

noisifier = AddNoise("RIRS_NOISES/noise_list_with_duration.csv", snr_low=0, snr_high=15)

for audio_file in tqdm(glob("accent_recognition/audio_files_treated/*.mp3")):
    audio = read_audio(audio_file)
    original_audio = Audio(audio, rate=48000)  # Assuming you want to process each audio file similarl
    noisy_audio = noisifier(audio.unsqueeze(0), torch.ones(1))
    noisy_path = audio_file.replace(".mp3", "_noisy.mp3").replace("accent_recognition/audio_files_treated", "accent_recognition/noisy_audio")
    write_audio(noisy_path, noisy_audio.squeeze(0), 48000)    
    # Save or process the noisy audio as needed
    # e.g., save to a new file or append to a list for further processing

  0%|          | 0/7623 [00:00<?, ?it/s]

100%|██████████| 7623/7623 [01:34<00:00, 80.60it/s]


In [74]:
import pandas as pd

train_df = pd.read_csv("accent_recognition/accent_train.tsv", sep="\t")
test_df = pd.read_csv("accent_recognition/accent_test.tsv", sep="\t")

In [ ]:
train_df_noisy = train_df.copy()
test_df_noisy = test_df.copy()
train_df_noisy['client_id'] = train_df_noisy['client_id'].apply(lambda x: x + "_noisy")
test_df_noisy['client_id'] = test_df_noisy['client_id'].apply(lambda x: x + "_noisy")
train_df_noisy['sentence_id'] = train_df_noisy['sentence_id'].apply(lambda x: x + "_noisy")
test_df_noisy['sentence_id'] = test_df_noisy['sentence_id'].apply(lambda x: x + "_noisy")
train_df_noisy['path'] = train_df_noisy['path'].apply(lambda x: x.replace(".mp3", "_noisy.mp3"))
test_df_noisy['path'] = test_df_noisy['path'].apply(lambda x: x.replace(".mp3", "_noisy.mp3"))
new_train_df = pd.concat([train_df, train_df_noisy], ignore_index=True)
new_train_df.sort_values(by=['sentence'], inplace=True)
new_train_df.tail()
new_test_df = pd.concat([test_df, test_df_noisy], ignore_index=True)
new_test_df.sort_values(by=['sentence'], inplace=True)
new_test_df.tail()

new_train_df.to_csv("accent_recognition/accent_train_noisy.tsv", sep="\t", index=False)
new_test_df.to_csv("accent_recognition/accent_test_noisy.tsv", sep="\t", index=False)

,client_id,sentence_id,path,sentence,age,gender,reduced_accents
673,7275df3ab29386b68c10c396e31ed2476d1e25de0ac979...,031ab8f4d0504c08c6d5582d8aec15d1e8dbdf5981b2e8...,common_voice_en_37281002.mp3,Your Bardic Inspiration die changes when you r...,thirties,male_masculine,North American
134,5506a0ac6b862d5ba2cb1b1f0bb4a388444ef7b1d87221...,cc841cb323de95557cfd3d48446b8aeebedaa87bc190e9...,common_voice_en_36955863.mp3,"‘An awful Sunday,’ commenced the paragraph ben...",twenties,male_masculine,African
1360,5506a0ac6b862d5ba2cb1b1f0bb4a388444ef7b1d87221...,cc841cb323de95557cfd3d48446b8aeebedaa87bc190e9...,common_voice_en_36955863_noisy.mp3,"‘An awful Sunday,’ commenced the paragraph ben...",twenties,male_masculine,African
2411,5ea73f02537215e9a8cb7be3d997d9574e72d85f082fb9...,5c63f223563400d318faf7c42f4423043707f2df7b2c45...,common_voice_en_22267156_noisy.mp3,‘This is insufferable!’ he exclaimed.,twenties,female_feminine,South Asian
1185,5ea73f02537215e9a8cb7be3d997d9574e72d85f082fb9...,5c63f223563400d318faf7c42f4423043707f2df7b2c45...,common_voice_en_22267156.mp3,‘This is insufferable!’ he exclaimed.,twenties,female_feminine,South Asian


In [1]:
import pandas as pd

In [4]:
train_df = pd.read_csv("accent_train_noisy.tsv", sep="\t")

In [6]:
train_split = train_df.sample(frac=0.8, random_state=42)

In [12]:
len(train_split) // (32*4)

52